# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [2]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests
import json

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

ModuleNotFoundError: No module named 'geoviews'

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [ ]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head()

## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [ ]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Weather Description"]
)

# Display the map
map_plot

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [ ]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Navirai"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Navirai"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Cidreira"] 
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Porto Belo"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Iguape"] 

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [ ]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
vacations_pd = vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end
itinerary_df = pd.concat(vacations_pd)

# Display sample data
itinerary_df.head()

## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [ ]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = pd.DataFrame(itinerary_df, columns = ["Lat", "Lng"])

# Display sample data
waypoints_df.head()

## Use GeoViews to create map that shows the four cities in the itinerary

In [ ]:
# Configure the map plot by using the itineraty_df
itinerary_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Weather Description"]
)

In [ ]:
# Display the route_map
itinerary_map

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [ ]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [ ]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [ ]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()
route_response

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [ ]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response['features'][0]['geometry']['coordinates']
display(legs)

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [ ]:
# Create and empty list to store the longitude of each step
lng = []

# Create and empty list to store the latitude of step
lat = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for steps in legs:
    for coordinates in steps:
        lat.append(coordinates[0])
        lng.append(coordinates[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [ ]:
# Create an empty DataFrame to store the steps' coordinates
route_df = []

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df = pd.DataFrame({'Latitude':(lat),'Longitude':(lng)})

# Display sample data
route_df.head()

##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [ ]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df)

In [ ]:
# Display a composed plot by using the route_map and route_path objects
route_path = gv.Path(route_df).opts(
    height=500,
    width=700,
    line_width = 3,
    color = "orange"
)

route_map = itinerary_map * route_path
route_map

In [ ]:
hvplot.save(route_map, 'Vacation_Itinerary/WeatherPy_travel_map.png')